In [144]:
import cv2
import numpy as np
from detection2 import Yaw


# name = "IMG_3603"
# name = "IMG_3468"

# name = "IMG_4295"
# name = "IMG_4296"
# name = "IMG_4297"
# name = "IMG_4299"
# name = "IMG_4300"
# name = "IMG_4301"
name = "IMG_4302"

cap = cv2.VideoCapture('sh/mov/'+ name +'.MOV')
frames = []
angles = []
yaw_obj = Yaw()


c= 1
while True:


    ret, frame = cap.read()
    if not ret:
        break
    yaww = yaw_obj.predict_yaw(frame)
    angles.append(yaww)
    frames.append(frame)

    if (c% 100 ==0):
        print(c , yaww)
    c+=1

    
cap.release()
video_array = np.array(frames)

Using cache found in C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-2 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)

Fusing layers... 


requirements: C:\Users\admin\.cache\torch\hub\requirements.txt not found, check failed.


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Model: "model_22"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_23 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_23[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                          

In [145]:
import os
import random
r = random.randint(0,12000)
# Path
# Create the directory
# 'GeeksForGeeks' in
# '/home / User / Documents'
os.mkdir("sh/mov/"+name+"__"+ str(r))

In [146]:
len(angles)

1745

In [147]:
def find_nearest(lst, num):
    if num is  None:
        return None

    nearest_num = lst[0]
    nearest_idx = 0

    if lst[0] is not None:
        min_diff = abs(num - lst[0])
    
    
    for i in range(1, len(lst) ):
        if lst[i] is  None:
            continue
        diff = abs(num - lst[i])
        if diff <= min_diff:
            nearest_num = lst[i]
            nearest_idx = i
            min_diff = diff
            
    return (nearest_num, nearest_idx)

In [148]:
def rotate_180(angle):
    return (angle + 180) % 360

In [149]:
def select_frame_from_period(frames ,angles , angle_range):
    print(angle_range ," -------> ", angles)
    a , b = angle_range
    angle_avg = (a+b) / 2
    angle , idx = find_nearest(angles , angle_avg)
    if angle is  None:
        return None
    if 280 > abs(angle - angle_avg) > 100:
        angle_avg = rotate_180(angle_avg)
        angle , idx = find_nearest(angles , angle_avg)
        angle = rotate_180(angle)
    return frames[idx] , angle





In [150]:
def frame_selection(angles , frames , num_frames = 30):
    selected_frames = []
    selected_angles = []
    angle_period = 360 // num_frames
    frame_period = frames.shape[0] // num_frames
    for i in range(num_frames):
        # print(i)
        half_period = angle_period // 2
        a = i * angle_period - half_period
        b = (i+1) * angle_period - half_period - 1
        c = i * frame_period
        d = (i+1) * frame_period -1
        
        frame_angle = select_frame_from_period( frames[c:d,:,:,:],angles[c:d] , angle_range = (a,b))

        if frame_angle is None:
            continue

        frame , angle = frame_angle
        
        
        selected_frames.append(frame)
        selected_angles.append(angle)

    return selected_frames , selected_angles




        

In [151]:
sel_frames , sel_angles = frame_selection(angles , video_array , 25)

(-7, 6)  ------->  [288.45631892249946, 286.9885295280552, 287.3258820832974, 287.6523464593357, 286.3243113230175, 287.58802312982743, 287.86820441995326, 286.86639545674086, 288.4138342768973, 287.4229083702451, 247.80157638432593, 285.05418596628374, 286.27019059223, 288.6124195367015, 288.88553072473474, 286.70960670073396, 286.74419052218855, 290.0064667674041, 290.31103611045546, 290.0323386794573, 288.8022015332914, 289.9243823783821, 287.742968587873, 286.87923707074583, 286.5369855605698, 287.8088560312605, 288.26888480427624, 288.599869059614, 289.118407815302, 287.48288086769645, 248.6205120897642, 249.03776166516454, 285.9314147958106, 285.01477576068106, 248.17495563900084, 248.46162134228857, 248.93926774079412, 285.7852751623011, 248.45158949835806, 249.02877825564653, 248.2946957281551, 248.86613687920183, 248.6059868335268, 249.08479436460644, 248.07105220287355, 247.94087472716183, 247.94080044882804, 247.4890771888128, 247.6468110725841, 288.9625573568678, 287.924460

In [152]:
sel_angles

[67.06853332662047,
 66.78531174745319,
 65.69432737566444,
 66.06662105675997,
 63.35343864285335,
 45.31852655351531,
 37.534126437179,
 191.80589031644547,
 176.7658090266154,
 163.09706329971635,
 136.3300648430966,
 138.45142332008112,
 124.92072878571827,
 116.36030705784339,
 240.31958330344492,
 215.0727695594765,
 214.8801000993825,
 198.24127347310105,
 350.9816970032326,
 332.9367491129211,
 312.8631420682914,
 295.3324952266595,
 294.3315545515559,
 248.16656901747496,
 248.07798143226415]

In [153]:
sel_frames = np.array(sel_frames)
size = sel_frames.shape[2] , sel_frames.shape[1]

fps = 2


out = cv2.VideoWriter('sh/mov/'+ name+"__"+ str(r) +'/reco_.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

for f in sel_frames:
    out.write(f)
out.release() 


In [154]:

# List of images
images = sel_frames # Replace with your list of NumPy arrays

# Write images to disk
for i, image in enumerate(images):
    filename = f"sh/mov/"+ name+"__"+ str(r) +f"/image_{i}.png"
    cv2.imwrite(filename, image)

## real time detection

In [155]:
def find_main_angle(angle , expectation_idx = 0): # expectation_idx is previous angle_idx

    # old mode
    # important_angles = [0 , 30 , 90  , 150 , 180 , 200 , 270 , 340 , 360]
    # angle_names = ["front" , "front-left" , "left" , "back-left" , "back" , "back-right" , "right" , "front-right" , "front"]
    
    # correct mode
    # important_angles = [90 , 30 , 0  , 340 , 270 , 200 , 180 , 150 , 90]
    # angle_names = ["left" , "front-left" , "front" , "front-right" , "right" , "back-right" , "back" , "back-left" , "left"]


    # temp mode
    important_angles = [ 270 , 200 , 180 , 150 , 90 , 30 , 0  , 340 , 270]
    angle_names = [ "right" , "back-right" , "back" , "back-left" , "left" , "front-left" , "front" , "front-right" , "right"]

    nearest = find_nearest(important_angles , angle)
    if nearest is None:
        return "------------" , expectation_idx # nothing detected
    
    ang , idx = nearest
    if 280 > abs(ang - important_angles[expectation_idx]) > 100: # dont mistake on 180 degree
        angle2 = rotate_180(angle)
        ang , idx = find_nearest(important_angles , angle2)
    if idx != expectation_idx + 1  : # dont turn back
        idx = expectation_idx
    return angle_names[idx] , idx
    
    


In [156]:
font = cv2.FONT_HERSHEY_SIMPLEX
x, y = 50, video_array[0].shape[0] - 100
color = (150, 35, 200)
font_scale = 2
thickness = 8



In [157]:
import cv2
import os
import copy
# Set the directory containing the images
li = copy.deepcopy(video_array)

# Create a window to display the video
cv2.namedWindow('Video', cv2.WINDOW_NORMAL)

# Loop through each image and display it in the window
c= 0 
expectation_idx = 0
yawObj = Yaw()

detected_video = []

for img in li:
    if c % 20 == 0 :
        yaww = yawObj.predict_yaw(img)
        ang , idx = find_main_angle(yaww , expectation_idx = expectation_idx)
        expectation_idx = idx
    cv2.putText(img, "angle: "+str(ang), (x,y), font, font_scale, color, thickness)
    cv2.imshow('Video', img)
    detected_video.append(img)
    c+=1
    # Wait for 25 milliseconds between each frame (40 frames per second)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the window and close all windows
cv2.destroyAllWindows()

Using cache found in C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-2 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients


requirements: C:\Users\admin\.cache\torch\hub\requirements.txt not found, check failed.


Adding AutoShape... 


Model: "model_23"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_24 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_24[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                          

In [158]:
detected_video = np.array(detected_video)
size = detected_video.shape[2] , detected_video.shape[1]

fps = 30


out = cv2.VideoWriter('sh/mov/RECO/___new3__'+ name+"_"+ str(r) +'reco.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

for f in detected_video:
    out.write(f)
out.release() 